# JUICE RPWI HF SID2 (RAW): L1a QL -- 2023/12/16

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import copy
import csv
import glob

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid2_lib as juice_sid2

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
unit_mode = 0                           # 0: sum      1: /Hz
cal_mode = 0                            # 0: raw      1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
f_mode = 1                              # 0: linear   1:log  in frequency
dump_mode = 0                           # 0: no-dump  1:plot dump
#
spec_mode = 1                           # 0: low-resoltion   1: high-resolution
ave_mode = 2                            # 0: simple sum      1: FFT sum     2: median sum   3: min sum
clean_mode = 0                          # 0: normal   1: cleaning in time   2: cleaning in time & frequency
time_mode = 1                           # 0: Epoch    1: data number

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 7.0                         # background: 7.5   CAL: 10
p_raw_min = 0.0                         # background: 2.5   CAL: 5

# *** Filter make ***
filter_mode = 0                         # 0: read     1: make

# *** Conversion factor: cal_mode ***
cf, p_max0, p_min0 = juice_cdf.cal_factors(0, cal_mode, cal, p_raw_max, p_raw_min)
print("conversion factor:", cf, "    MAX-min:", p_max0, p_min0)

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
# *** Ground - Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name_list = ['SID02_20231007-0349.cdf',
                  'SID02_20231117-1607.cdf',
                 ]
"""
data_name_list = ['SID02-high_20231019-1749.cdf',
                  'SID02-high_20231020-1218.cdf',
                  'SID02-high_20231020-1222.cdf',
                 ]

# 202312 -- TEST
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202312_FS/cdf/'    # 202312_FS test folder
data_name_list = ['SID2_20000101T004732-20000101T004929_0.cdf',
                  'SID2_20000101T004934-20000101T005147_1.cdf',
                  'SID2_20000101T005151-20000101T005357_2.cdf',
                  'SID2_20000101T005401-20000101T005538_3.cdf',
                  'SID2_20000101T005543-20000101T005624_4.cdf',
                 ]
"""

In [ ]:
# *** Flight - Ver.1
"""
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2023/'        # CDF data folder

# *** 20230419 ***
data_name_list = [#'04/19/SID2_20230419T135849-20230419T141229.cdf',
                  '04/19/SID2_20230419T141231-20230419T141402.cdf',
                 ]

# *** 20230530 ***
data_name_list = ['05/30/SID2_20230530T100326-20230530T100925.cdf',
                  '05/30/SID2_20230530T100927-20230530T100937.cdf',
                 ]

# *** 20230601 ***
data_name_list = ['06/01/SID2_20230601T120759-20230601T120857.cdf',
                  '06/01/SID2_20230601T121435-20230601T121533.cdf',
                  '06/01/SID2_20230601T122138-20230601T122236.cdf',
                  '06/01/SID2_20230601T122707-20230601T122805.cdf',
                 ]

# *** 20230712-13 ***
data_name_list = ['07/12/SID2_20230712T090434-20230712T093848.cdf',
                  '07/12/SID2_20230712T093942-20230712T101355.cdf',
                  '07/12/SID2_20230712T101449-20230712T104147.cdf',
                 ]
data_name_list = ['07/12/SID2_20230712T104149-20230712T232406.cdf']
#
data_name_list = ['07/12/SID2_20230712T232408-20230712T235156.cdf']
#      type-III
data_name_list = ['07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                 ]
#      battery
data_name_list = ['07/13/SID2_20230713T004648-20230713T011342.cdf',
                  '07/13/SID2_20230713T011436-20230713T014134.cdf',
                  '07/13/SID2_20230713T014136-20230713T020924.cdf',
                  '07/13/SID2_20230713T020928-20230713T023718.cdf',
                  '07/13/SID2_20230713T023720-20230713T030416.cdf',
                  '07/13/SID2_20230713T030510-20230713T033208.cdf',
                  '07/13/SID2_20230713T033210-20230713T040000.cdf',
                  '07/13/SID2_20230713T040002-20230713T042751.cdf',
                  '07/13/SID2_20230713T042753-20230713T045449.cdf',
                  '07/13/SID2_20230713T045543-20230713T050917.cdf',
                 ]
"""

In [ ]:
# *** Ground - Ver.1
"""
# *** 202105 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202105/'        # CDF data folder
data_name_list = ['SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32770.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32774.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32775.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32776.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32777.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day3_xid32778.cdf',
                 ]
data_name_list = ['SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32772.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32773.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32774.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32775.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32776.cdf',
                  'SID2_20210531_SCPFM_PTR_RPWI_2_day5_xid32777.cdf',
                 ]

# *** 202106 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202106/'        # CDF data folder
data_name_list = ['SID2_SCTBTV_Phase11_xid32831.cdf',
                  'SID2_SCTBTV_Phase11_xid32832.cdf',
                  'SID2_SCTBTV_Phase11_xid32833.cdf',
                  'SID2_SCTBTV_Phase11_xid32834.cdf',
                 ]
data_name_list = ['SID2_SCTBTV_Phase13_xid32844.cdf',
                  'SID2_SCTBTV_Phase13_xid32845.cdf',
                  'SID2_SCTBTV_Phase13_xid32846.cdf',
                 ]

# *** 202111 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202111/'        # CDF data folder
data_name_list = ['SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32770.cdf',
                  'SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32771.cdf',
                  'SID2_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32772.cdf',
                 ]
data_name_list = ['SID2_SCPFM_RPWI_30c_xid32776.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32777.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32778.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32779.cdf',
                 ]

# *** 202207 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202207/'        # CDF data folder
data_name_list = ['SID2_SCPFM_RPWI_30c_xid32776.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32777.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32778.cdf',
                  'SID2_SCPFM_RPWI_30c_xid32779.cdf',
                 ]

# *** 202208 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202208/'        # CDF data folder
data_name_list = ['SID2_20220824_HF-FFT-rerun_xid32791.cdf',
                  'SID2_20220824_HF-FFT-rerun_xid32792.cdf',
                  'SID2_20220824_HF-FFT-rerun_xid32793.cdf',
                  ]
"""

In [ ]:
# *** Group read
"""
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID02*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    cdf = pycdf.CDF(cdf_file)
    data1 = juice_sid2.hf_sid2_read(cdf, cf)

    if i==0:
        data = data1
        print(data.Eu_i.shape)
    else:
        data = juice_sid2.hf_sid2_add(data, data1)
        print(data.Eu_i.shape)

print(data_name);  data_name = os.path.split(data_name)[1];  print(data_name)

In [ ]:
data = juice_sid2.hf_sid2_proc(data)

date1 = data.epoch[0]
date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]
date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

n_time0 = data.Eu_i.shape[0]
n_freq0 = data.Eu_i.shape[1]
n_samp0 = data.Eu_i.shape[2]
f_min0 = data.frequency[0][0][0]
f_max0 = (max(np.ravel(data.frequency)))

In [ ]:
# Mode 
N_ch0 = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
print("Ch:", N_ch0, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("Num-samples:", n_time0, "   Num-Frequency", n_freq0, "   Length:", n_samp0)
print("Frequency, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0][0], data.freq_step[0][0][0])
print("Time-length:", data.time[0][0][n_samp0-1], "sec in 1-sweep")

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 14))
ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3)
ax4 = fig.add_subplot(6, 1, 4);  ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6)

ax1.plot(np.ravel(data.Eu_i[:][:]), '-r', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(data.Eu_q[:][:]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[:][:]), '-r', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(data.Ev_q[:][:]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[:][:]), '-r', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(data.Ew_q[:][:]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency),  '-b', linewidth=.5, label='Frequency');
ax4.plot(np.ravel(data.freq_step*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width*10), ':b', linewidth=1.0, label='width*10')
ax4.plot(np.ravel(data.sweep_start)*data.frequency[0][-1][0], '-r', linewidth=.5, label='Sweep Start');
ax5.plot(np.ravel(data.T_HF_FPGA),  ':r', label='T (HK-FPGA)')
ax5.plot(np.ravel(data.T_RWI_CH1),  ':b', label='T (RWI1)')
ax5.plot(np.ravel(data.T_RWI_CH2),  ':g', label='T (RWI2)')
ax6.plot(np.ravel(data.epoch[:]), '.')

ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  ax5.set_ylabel('T [degC]');  
ax6.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
"""
# range: Y-axis
ylim=[-10**(p_max0-3), 10**(p_max0-3)]
ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
ylim=[f_min0, f_max0]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

## RAW - First 2 Sweeps

In [ ]:
n_sweep0 = 0;    n_sweep1 = n_sweep0 + 2
print("[specific sweep]  SWEEP:", n_sweep0, n_sweep1, "in total of", n_time0)
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(data.Eu_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Eu_i'); 
ax1.plot(np.ravel(data.Eu_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ev_i'); 
ax2.plot(np.ravel(data.Ev_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ew_i'); 
ax3.plot(np.ravel(data.Ew_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep0:n_sweep1]),  '-b', linewidth=.5, label='Frequency');
ax4.plot(np.ravel(data.sweep_start[n_sweep0:n_sweep1])*data.frequency[n_sweep0][-1][0], '-r', label='Sweep Start');
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  
date1 = data.epoch[n_sweep0];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[len(np.ravel(data.Eu_i[n_sweep0:n_sweep1]))//2 - 32, len(np.ravel(data.Eu_i[n_sweep0:n_sweep1]))//2 + 32];  print(xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_max0), 10**(p_max0)]; ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
"""
ylim=[f_min, f_max]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

## Raw - First 3 steps

In [ ]:
n_sweep = n_time0//2;  n_step0 = 0;  n_step1 = n_step0 + 3
print("[specific sweep]  SWEEP:", n_sweep, "in total of", n_time0,  "[Steps]", n_step0, "-", n_step1)

fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(data.Eu_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Eu_i')
ax1.plot(np.ravel(data.Eu_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ev_i')
ax2.plot(np.ravel(data.Ev_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ew_i')
ax3.plot(np.ravel(data.Ew_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep][n_step0:n_step1]),  '-b', linewidth=.5, label='Frequency')
ax4.plot(np.ravel(data.sweep_start[n_sweep][n_step0:n_step1])*data.frequency[n_sweep][n_step1][0], '-r', label='Sweep Start')
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  
date1 = data.epoch[n_sweep];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""
#ylim=[f_min, f_max]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-step.png'
    fig.savefig(png_fname)

# Get Spectrum

In [ ]:
# Spec mode: high resolution  
spec = juice_sid2.hf_sid2_getspec(data)
power_str = juice_cdf.power_label(cal_mode, unit_mode)
#
EE_2d   = spec.EE.transpose();  EuEu_2d = spec.EuEu.transpose();  EvEv_2d = spec.EvEv.transpose();  EwEw_2d = spec.EwEw.transpose()
"""
E_DoLuv_2d = spec.E_DoLuv.transpose()
E_DoLvw_2d = spec.E_DoLvw.transpose()
E_DoLwu_2d = spec.E_DoLwu.transpose()
E_DoCuv_2d = spec.E_DoCuv.transpose()
E_DoCvw_2d = spec.E_DoCvw.transpose()
E_DoCwu_2d = spec.E_DoCwu.transpose()
E_ANGuv_2d = spec.E_ANGuv.transpose()
E_ANGvw_2d = spec.E_ANGvw.transpose()
E_ANGwu_2d = spec.E_ANGwu.transpose()
"""
#
f_min = spec.freq[0][0]
f_max = spec.freq[0][-1]
d_freq = spec.freq[0][1] - spec.freq[0][0]
freq_1d = spec.freq[0]
# freq_width_1d  = spec.freq_width[0]
Epoch_1d       = data.epoch.tolist()

n_time1 = spec.EE.shape[0]
n_freq1 = spec.EE.shape[1]
power_str = juice_cdf.power_label(cal_mode, unit_mode)

"""
freq_width_2d  = data.freq_width.transpose()
if unit_mode == 1:
    EuEu_2d = EuEu_2d / freq_width_2d / 1000
    EvEv_2d = EvEv_2d / freq_width_2d / 1000
    EwEw_2d = EwEw_2d / freq_width_2d / 1000
"""
#
print(power_str, "EuEu:", spec.EuEu.shape, f_min, "-", f_max, "(df =", d_freq, ") kHz")


# Spectrum

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
# n_sweep1 = 50;  n_sweep2 = 51;  n_sweep3 = 52
# Y-range
p_min = p_min0-2;   p_max = p_max0
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_min0;  f_max = 5000
# f_mode = 0;   f_min = 5000;  f_max = 10000;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d, spec.EE[n_sweep1], '-r', linewidth=.5, label='EE 1st')
ax2.plot(freq_1d, spec.EuEu[n_sweep1], '-r', linewidth=.5, label='EuEu 1st')
ax3.plot(freq_1d, spec.EvEv[n_sweep1], '-r', linewidth=.5, label='EvEv 1st')
ax4.plot(freq_1d, spec.EwEw[n_sweep1], '-r', linewidth=.5, label='EwEw 1st')
ax1.plot(freq_1d, spec.EE[n_sweep2], '-g', linewidth=.5, label='EE mid')
ax2.plot(freq_1d, spec.EuEu[n_sweep2], '-g', linewidth=.5, label='EuEu mid')
ax3.plot(freq_1d, spec.EvEv[n_sweep2], '-g', linewidth=.5, label='EvEv mid')
ax4.plot(freq_1d, spec.EwEw[n_sweep2], '-g', linewidth=.5, label='EwEw mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], '-b', linewidth=.5, label='EE last')
ax2.plot(freq_1d, spec.EuEu[n_sweep3], '-b', linewidth=.5, label='EuEu last')
ax3.plot(freq_1d, spec.EvEv[n_sweep3], '-b', linewidth=.5, label='EvEv last')
ax4.plot(freq_1d, spec.EwEw[n_sweep3], '-b', linewidth=.5, label='EwEw last')
ax1.set_yscale('log'); ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax4.set_xlabel('Frequency [kHz]')
#
date1 = data.epoch[n_sweep1];  date1 = date1.strftime('First: %Y/%m/%d %R:%S   ')
date2 = data.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = data.epoch[n_sweep3];  date3 = date3.strftime('Last: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
# xlim=[8150, 8250]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
"""
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

# Peak
print("[First]")
peak_E = np.ravel(spec.EE[n_sweep1]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print("[Mid]")
peak_E = np.ravel(spec.EE[n_sweep2]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print("[Last]")
peak_E = np.ravel(spec.EE[n_sweep3]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")

# Frequency - Time diagram

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
if f_mode == 0:
    f_min = f_min0;  f_max = 5000
# Z-range
p_min = p_min0+1;    p_max = p_max0-3
#f_mode = 0;   f_min = 1200;  f_max = 1600;  print(data.frequency[0][0][0], data.frequency[0][8][0])

# Epoch_1d
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(5, 1, 1);  ax2 = fig2d.add_subplot(5, 1, 2); ax3 = fig2d.add_subplot(5, 1, 3); ax4 = fig2d.add_subplot(5, 1, 4)
if time_mode == 1:
    ax5 = fig2d.add_subplot(5, 1, 5)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
if time_mode == 1:
    ax5.set_xlabel(str_date)
else:
    ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p5 = ax5.plot(np.ravel(data.epoch[:]), '.')
    pp4 = fig2d.colorbar(p4, ax=ax5, orientation="vertical"); pp4.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)

# range: Y-axis
"""
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)
"""

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

# Median Filter

In [ ]:
df = 13    # frequency filter width / 2
dt = 13    # time filter width / 2

spec_org = copy.deepcopy(spec)
spec_org.EE = 10 * np.log10(spec_org.EE)
spec_org.EuEu = 10 * np.log10(spec_org.EuEu)
spec_org.EvEv = 10 * np.log10(spec_org.EvEv)
spec_org.EwEw = 10 * np.log10(spec_org.EwEw)
EE_med   = np.median(spec_org.EE, axis=0)
EuEu_med = np.median(spec_org.EuEu, axis=0)
EvEv_med = np.median(spec_org.EvEv, axis=0)
EwEw_med = np.median(spec_org.EwEw, axis=0)
#
spec_cln = copy.deepcopy(spec_org)
spec_bgd = copy.deepcopy(spec_org)
spec_dif = copy.deepcopy(spec_org)

# Noise cleaning -- in frequency
print(n_time0,n_freq1,freq_1d.shape)
for i in range(n_time0):
    for j in range(n_freq1):
        j1 = j - df
        if j1 < 0:
            j1 = 0
        j2 = j + df
        if j2 > n_freq1-1:
            j2 = n_freq1-1
        spec_cln.EE[i, j]   = np.median(spec_org.EE[i, j1:j2])
        spec_cln.EuEu[i, j] = np.median(spec_org.EuEu[i, j1:j2])
        spec_cln.EvEv[i, j] = np.median(spec_org.EvEv[i, j1:j2])
        spec_cln.EwEw[i, j] = np.median(spec_org.EwEw[i, j1:j2])
EE_cln_med   = np.median(spec_cln.EE, axis=0)
EuEu_cln_med = np.median(spec_cln.EuEu, axis=0)
EvEv_cln_med = np.median(spec_cln.EvEv, axis=0)
EwEw_cln_med = np.median(spec_cln.EwEw, axis=0)

# Background -- in time
for i in range(n_time0):
    for j in range(n_freq1):
        i1 = i - dt
        if i1 < 0:
            i1 = 0
        i2 = i + dt
        if i2 > n_time0-1:
            i2 = n_time0-1
        spec_bgd.EE[i, j]   = np.median(spec_cln.EE[i1:i2, j])
        spec_bgd.EuEu[i, j] = np.median(spec_cln.EuEu[i1:i2, j])
        spec_bgd.EvEv[i, j] = np.median(spec_cln.EvEv[i1:i2, j])
        spec_bgd.EwEw[i, j] = np.median(spec_cln.EwEw[i1:i2, j])
EE_bgd_med = np.median(spec_bgd.EE, axis=0)
EuEu_bgd_med = np.median(spec_bgd.EuEu, axis=0)
EvEv_bgd_med = np.median(spec_bgd.EvEv, axis=0)
EwEw_bgd_med = np.median(spec_bgd.EwEw, axis=0)

# Diff
spec_dif.EE = spec_org.EE - spec_bgd.EE
spec_dif.EuEu = spec_org.EuEu - spec_bgd.EuEu
spec_dif.EvEv = spec_org.EvEv - spec_bgd.EvEv
spec_dif.EwEw = spec_org.EwEw - spec_bgd.EwEw
EE_dif_med = np.median(spec_dif.EE, axis=0)
EuEu_dif_med = np.median(spec_dif.EuEu, axis=0)
EvEv_dif_med = np.median(spec_dif.EvEv, axis=0)
EwEw_dif_med = np.median(spec_dif.EwEw, axis=0)

# 2D
EE_cln_2d   = spec_cln.EE.transpose() 
EuEu_cln_2d = spec_cln.EuEu.transpose()
EvEv_cln_2d = spec_cln.EvEv.transpose()
EwEw_cln_2d = spec_cln.EwEw.transpose()

# Spectrum - median

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0-2;   p_max = p_max0
# X-range
f_min = f_min0;  f_max = f_max0
#if f_mode == 0:
#    f_min = f_min0;  f_max = 5000
#f_mode = 0;   f_min = 2000;  f_max = 5000;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(freq_1d, EE_med, '-k', linewidth=.5, label='EE ave')
ax2.plot(freq_1d, EuEu_med, '-k', linewidth=.5, label='EuEu ave')
ax3.plot(freq_1d, EvEv_med, '-k', linewidth=.5, label='EvEv ave')
ax4.plot(freq_1d, EwEw_med, '-k', linewidth=.5, label='EwEw ave')
ax1.plot(freq_1d, EE_cln_med, '-r', linewidth=.5, label='EE f-clean')
ax2.plot(freq_1d, EuEu_cln_med, '-r', linewidth=.5, label='EuEu f-clean')
ax3.plot(freq_1d, EvEv_cln_med, '-r', linewidth=.5, label='EvEv f-clean')
ax4.plot(freq_1d, EwEw_cln_med, '-r', linewidth=.5, label='EwEw f-clean')
ax1.plot(freq_1d, EE_bgd_med, '-g', linewidth=.5, label='EE BG')
ax2.plot(freq_1d, EuEu_bgd_med, '-g', linewidth=.5, label='EuEu BG')
ax3.plot(freq_1d, EvEv_bgd_med, '-g', linewidth=.5, label='EvEv BG')
ax4.plot(freq_1d, EwEw_bgd_med, '-g', linewidth=.5, label='EwEw BG')
ax1.plot(freq_1d, EE_dif_med, '-b', linewidth=.5, label='EE Diff')
ax2.plot(freq_1d, EuEu_dif_med, '-b', linewidth=.5, label='EuEu Diff')
ax3.plot(freq_1d, EvEv_dif_med, '-b', linewidth=.5, label='EvEv Diff')
ax4.plot(freq_1d, EwEw_dif_med, '-b', linewidth=.5, label='EwEw Diff')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Peak
from scipy.signal import find_peaks
EE_dif_med_peaks,_ = find_peaks(EE_dif_med, height=3.0)
EuEu_dif_med_peaks,_ = find_peaks(EuEu_dif_med, height=3.0)
EvEv_dif_med_peaks,_ = find_peaks(EvEv_dif_med, height=3.0)
EwEw_dif_med_peaks,_ = find_peaks(EwEw_dif_med, height=3.0)
ax1.plot(freq_1d[EE_dif_med_peaks], EE_dif_med[EE_dif_med_peaks], '.r', label='EE Diff peak')
ax2.plot(freq_1d[EuEu_dif_med_peaks], EuEu_dif_med[EuEu_dif_med_peaks], '.r', label='EuEu Diff peak')
ax3.plot(freq_1d[EvEv_dif_med_peaks], EvEv_dif_med[EvEv_dif_med_peaks], '.r', label='EvEv Diff peak')
ax4.plot(freq_1d[EwEw_dif_med_peaks], EwEw_dif_med[EwEw_dif_med_peaks], '.r', label='EwEw Diff peak')

# Label
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax4.set_xlabel('Frequency [kHz]')
#
date1 = data.epoch[n_sweep1];  date1 = date1.strftime('First: %Y/%m/%d %R:%S   ')
date2 = data.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = data.epoch[n_sweep3];  date3 = date3.strftime('Last: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
# xlim=[8150, 8250]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
#ylim=[p_min*10, p_max*10]
ylim=[-10, 50]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec_med.png'
    fig.savefig(png_fname)

"""
# Peak
print("[First]")
peak_E = np.ravel(spec.EE[n_sweep1]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print("[Mid]")
peak_E = np.ravel(spec.EE[n_sweep2]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print("[Last]")
peak_E = np.ravel(spec.EE[n_sweep3]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
"""

# FT - median

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
if f_mode == 0:
    f_min = f_min0;  f_max = 5000
# Z-range
p_min = p_min0+1;    p_max = p_max0-3
#f_mode = 0;   f_min = 1200;  f_max = 1600;  print(data.frequency[0][0][0], data.frequency[0][8][0])

# Epoch_1d
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2); ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape, EE_cln_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_cln_2d,   norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EuEu_cln_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EvEv_cln_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p4 = ax4.pcolormesh(num_1d, freq_1d, EwEw_cln_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_cln_2d,   norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EuEu_cln_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EvEv_cln_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d, EwEw_cln_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT_med.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log_med.png'
    fig2d.savefig(png_fname)

# MASK formation

### MASK formation: from natural data

In [ ]:
# >1dB
mask_index = np.where(EE_dif_med >= 1)

n_mask = 19392
mask_list0 = np.zeros(n_mask * 3)
mask_list0 = mask_list0.reshape(n_mask, 3)
mask_dir = './lib/'
mask_name = 'hf_mask_natural.csv'
mask_file = mask_dir + mask_name

if filter_mode > 0:
    for i in range(n_mask):
        mask_list0[i, 0] = 80.0 + 2.3125 * i
        mask_list0[i, 1] = 80.0 + 2.3125 * (i+1)
        mask_list0[i, 2] = 0

    for j in range( len(mask_index[0]) ):
        f0 = freq_1d[ mask_index[0][j] ]
        f1 = np.int16((f0 - 80.0) // 2.3125)
        if f1 >= 0 and f1 < n_mask-2:
            mask_list0[f1-1, 2] = 1
            mask_list0[f1, 2] = 1
            mask_list0[f1+1, 2] = 1

    # mask_artificial
    with open(mask_file, 'w') as f:
        writer = csv.writer(f)
        for i in range(n_mask):
            writer.writerow([ i+1, mask_list0[i, 0], mask_list0[i, 1], mask_list0[i, 2] ])
    print("mask_natural made: ", mask_list0.shape, mask_file)

else:
    # mask_natural read
    with open(mask_file, 'r') as f:
        reader = csv.reader(f)
        i = 0
        for row in reader:
            mask_list0[i, 0] = row[1]
            mask_list0[i, 1] = row[2]
            mask_list0[i, 2] = row[3]
            i = i+1
    print("mask_natural read: ", mask_list0.shape, mask_file)

### MASK formation: harmonics of 200.4 kHz

In [ ]:
n_mask = 19392
mask_list = np.zeros(n_mask * 3)
mask_list = mask_list.reshape(n_mask, 3)
mask_dir = './lib/'
mask_name = 'hf_mask_artificial.csv'
mask_file = mask_dir + mask_name

if filter_mode > 0:
    # mask_artificiial make
    f_mask = 200.4
    f1_mask = 200.4 * 0.35
    f2_mask = 200.4 * 0.66
    df1_mask = 17.0
    df2_mask = 13.0

    for i in range(n_mask):
        mask_list[i, 0] = 80.0 + 2.3125 * i
        mask_list[i, 1] = 80.0 + 2.3125 * (i+1)
        mask_list[i, 2] = 0
        #
        f0_0 = (mask_list[i, 0] // f_mask) * f_mask;  f0_1 = mask_list[i, 0] - f0_0   ## lower-end
        f1_0 = (mask_list[i, 1] // f_mask) * f_mask;  f1_1 = mask_list[i, 1] - f1_0   ## upper-end
        # 200.4 * N kHz harmonics
        if (f0_1 > f_mask-df1_mask or f0_1 < df2_mask or f1_1 > f_mask-df1_mask or f1_1 < df2_mask):
            mask_list[i, 2] = 1
        # 200.4 * (N + 0.36) kHz harmonics
        if ((f0_1 > f1_mask-df1_mask and f0_1 < f1_mask+df2_mask) or (f1_1 > f1_mask-df1_mask and f1_1 < f1_mask+df2_mask)):
            mask_list[i, 2] = 2
        # 200.4 * (N + 0.65) kHz harmonics
        if ((f0_1 > f2_mask-df1_mask and f0_1 < f2_mask+df2_mask) or (f1_1 > f2_mask-df1_mask and f1_1 < f2_mask+df2_mask)):
            mask_list[i, 2] = 3

    with open(mask_file, 'w') as f:
        writer = csv.writer(f)
        for i in range(n_mask):
            writer.writerow([ i+1, mask_list[i, 0], mask_list[i, 1], mask_list[i, 2] ])
    print("mask_artificial made: ", mask_list.shape, mask_file)

else:
    # mask_artificiial read
    with open(mask_file, 'r') as f:
        reader = csv.reader(f)
        i = 0
        for row in reader:
            mask_list[i, 0] = row[1]
            mask_list[i, 1] = row[2]
            mask_list[i, 2] = row[3]
            i = i+1
    print("mask_artificial read: ", mask_list.shape, mask_file)

### Mask filter - natural

In [ ]:
spec_msk0 = copy.deepcopy(spec)
spec_msk0.EE = 10 * np.log10(spec_msk0.EE)

for i in range(freq_1d.shape[0]):
    num_mask = np.int16((freq_1d[i] - 80) // 2.3125)
    if (num_mask >= 0 and num_mask < n_mask):
        if mask_list0[num_mask,2] > 0:
            spec_msk0.EE[:,i-1] = 0
            spec_msk0.EE[:,i] = 0
            if i<freq_1d.shape[0]-1:
                spec_msk0.EE[:,i+1] = 0

# delete
mask_index0    = np.where(spec_msk0.EE[0] == 0)
spec_msk0.EE   = np.delete(spec_msk0.EE, mask_index0, 1)
spec_msk0.EuEu = np.delete(spec_msk0.EuEu, mask_index0, 1)
spec_msk0.EvEv = np.delete(spec_msk0.EvEv, mask_index0, 1)
spec_msk0.EwEw = np.delete(spec_msk0.EwEw, mask_index0, 1)
freq_1d_msk0   = np.delete(freq_1d, mask_index0, 0)

# Median
EE_msk0   = np.median(spec_msk0.EE, axis=0)
EuEu_msk0 = np.median(spec_msk0.EuEu, axis=0)
EvEv_msk0 = np.median(spec_msk0.EvEv, axis=0)
EwEw_msk0 = np.median(spec_msk0.EwEw, axis=0)

# 2D
EE_msk0_2d = spec_msk0.EE.transpose() 
EuEu_msk0_2d = spec_msk0.EuEu.transpose() 
EvEv_msk0_2d = spec_msk0.EvEv.transpose() 
EwEw_msk0_2d = spec_msk0.EwEw.transpose() 

### Mask filter - artificial

In [ ]:
spec_msk = copy.deepcopy(spec)
spec_msk.EE = 10 * np.log10(spec_msk.EE)

for i in range(freq_1d.shape[0]):
    num_mask = np.int16((freq_1d[i] - 80) // 2.3125)
    if (num_mask >= 0 and num_mask < n_mask):
        if mask_list[num_mask,2] > 0:
            spec_msk.EE[:,i] = 0

# delete
mask_index    = np.where(spec_msk.EE[0] == 0)
spec_msk.EE   = np.delete(spec_msk.EE, mask_index, 1)
spec_msk.EuEu = np.delete(spec_msk.EuEu, mask_index, 1)
spec_msk.EvEv = np.delete(spec_msk.EvEv, mask_index, 1)
spec_msk.EwEw = np.delete(spec_msk.EwEw, mask_index, 1)
freq_1d_msk   = np.delete(freq_1d, mask_index, 0)

# Median
EE_msk   = np.median(spec_msk.EE, axis=0)
EuEu_msk = np.median(spec_msk.EuEu, axis=0)
EvEv_msk = np.median(spec_msk.EvEv, axis=0)
EwEw_msk = np.median(spec_msk.EwEw, axis=0)

# 2D
EE_msk_2d   = spec_msk.EE.transpose() 
EuEu_msk_2d = spec_msk.EuEu.transpose() 
EvEv_msk_2d = spec_msk.EvEv.transpose() 
EwEw_msk_2d = spec_msk.EwEw.transpose() 

# Spectrum - mask cleaned

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0-2;   p_max = p_max0
# X-range
f_mode = 0;  f_min = 0;  f_max = 5000

fig = plt.figure(figsize=(12, 20))
ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
ax5 = fig.add_subplot(9, 1, 5);  ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8)
ax9 = fig.add_subplot(9, 1, 9)

ax1.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax1.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax1.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax2.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax2.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax2.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax3.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax3.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax3.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax4.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax4.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax4.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax5.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax5.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax5.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax6.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax6.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax6.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax7.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax7.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax7.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax8.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax8.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax8.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')
ax9.plot(freq_1d, EE_med, ':g', linewidth=.5, label='EE ave')
ax9.plot(freq_1d_msk0, EE_msk0, '-b', linewidth=1, label='EE mask-nat')
#ax9.plot(freq_1d_msk, EE_msk, '-r', linewidth=1, label='EE mask-art')

# Mask - natural
ax1.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax2.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax3.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax4.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax5.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax6.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax7.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax8.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
ax9.plot(mask_list0[:,0], mask_list0[:,2]*15, '.r', label='mask-nat')
## Mask - artificial
#ax1.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax2.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax3.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax4.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax5.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax6.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax7.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax8.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')
#ax9.plot(mask_list[:,0], mask_list[:,2]*10, '.r', label='mask-art')

# Label
# ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); 
ax9.set_xlabel('Frequency [kHz]')
#
date1 = data.epoch[n_sweep1];  date1 = date1.strftime('First: %Y/%m/%d %R:%S   ')
date2 = data.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = data.epoch[n_sweep3];  date3 = date3.strftime('Last: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)

ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);  ax9.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min,       f_max];        ax1.set_xlim(xlim); 
xlim=[f_min+5000,  f_max+5000];   ax2.set_xlim(xlim);  
xlim=[f_min+10000, f_max+10000];  ax3.set_xlim(xlim); 
xlim=[f_min+15000, f_max+15000];  ax4.set_xlim(xlim)
xlim=[f_min+20000, f_max+20000];  ax5.set_xlim(xlim)
xlim=[f_min+25000, f_max+25000];  ax6.set_xlim(xlim)
xlim=[f_min+30000, f_max+30000];  ax7.set_xlim(xlim)
xlim=[f_min+35000, f_max+35000];  ax8.set_xlim(xlim)
xlim=[f_min+40000, f_max+40000];  ax9.set_xlim(xlim)

# range: Y-axis
#ylim=[p_min*10, p_max*10]
ylim=[1, 50]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim); ax5.set_ylim(ylim); ax6.set_ylim(ylim);  
ax7.set_ylim(ylim); ax8.set_ylim(ylim);  ax9.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_mask.png'
    fig.savefig(png_fname)

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
if f_mode == 0:
    f_min = f_min0;  f_max = 5000
# Z-range
p_min = p_min0+1;    p_max = p_max0-3
f_mode = 1;   f_min = 80;  f_max = 45000

# Epoch_1d
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[12,16])
ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax4 = fig2d.add_subplot(3, 1, 3)
# ax3 = fig2d.add_subplot(4, 1, 3);  

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log');        ax4.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]'); ax4.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EE original');  ax2.set_title('EE median in frequency');  ax4.set_title('EE mask - natural')
#ax3.set_title('EE mask - artiticial');  
ax4.set_xlabel(str_date)

# Plot
print(num_1d.shape, freq_1d.shape)  #, EE_msk_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d,   freq_1d, EE_2d,           norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d,   freq_1d, EE_cln_2d,       norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    # p3 = ax3.pcolormesh(num_1d,   freq_1d_msk, EE_msk_2d,   norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p4 = ax4.pcolormesh(num_1d,   freq_1d_msk0, EE_msk0_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
else:
    p1 = ax1.pcolormesh(num_1d,   freq_1d, EE_2d,           norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d,   freq_1d, EE_cln_2d,       norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    # p3 = ax3.pcolormesh(Epoch_1d, freq_1d_msk, EE_msk_2d,   norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
    p4 = ax4.pcolormesh(Epoch_1d, freq_1d_msk0, EE_msk0_2d, norm=colors.Normalize(vmin=p_min*10, vmax=p_max*10), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
# pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT_msk.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log_msk.png'
    fig2d.savefig(png_fname)

### Median Filter

In [ ]:
df = 13    # frequency filter width / 2
dt = 13    # time filter width / 2

spec_org = copy.deepcopy(spec_msk0)
spec_org.EE = 10 * np.log10(spec_org.EE)
spec_org.EuEu = 10 * np.log10(spec_org.EuEu)
spec_org.EvEv = 10 * np.log10(spec_org.EvEv)
spec_org.EwEw = 10 * np.log10(spec_org.EwEw)
EE_med   = np.median(spec_org.EE, axis=0)
EuEu_med = np.median(spec_org.EuEu, axis=0)
EvEv_med = np.median(spec_org.EvEv, axis=0)
EwEw_med = np.median(spec_org.EwEw, axis=0)
#
spec_cln = copy.deepcopy(spec_org)
spec_bgd = copy.deepcopy(spec_org)
spec_dif = copy.deepcopy(spec_org)

# Noise cleaning -- in frequency
print(spec_msk0.EE.shape)
print(len(freq_1d_msk0))
n_freq1_msk = spec_org.EE.shape[1]
print(n_time0, n_freq1_msk, freq_1d_msk0.shape)
for i in range(n_time0):
    for j in range(n_freq1_msk):
        j1 = j - df
        if j1 < 0:
            j1 = 0
        j2 = j + df
        if j2 > n_freq1_msk-1:
            j2 = n_freq1_msk-1
        spec_cln.EE[i, j]   = np.median(spec_org.EE[i, j1:j2])
        spec_cln.EuEu[i, j] = np.median(spec_org.EuEu[i, j1:j2])
        spec_cln.EvEv[i, j] = np.median(spec_org.EvEv[i, j1:j2])
        spec_cln.EwEw[i, j] = np.median(spec_org.EwEw[i, j1:j2])
EE_cln_med   = np.median(spec_cln.EE, axis=0)
EuEu_cln_med = np.median(spec_cln.EuEu, axis=0)
EvEv_cln_med = np.median(spec_cln.EvEv, axis=0)
EwEw_cln_med = np.median(spec_cln.EwEw, axis=0)

# Background -- in time
for i in range(n_time0):
    for j in range(n_freq1_msk):
        i1 = i - dt
        if i1 < 0:
            i1 = 0
        i2 = i + dt
        if i2 > n_time0-1:
            i2 = n_time0-1
        spec_bgd.EE[i, j]   = np.median(spec_cln.EE[i1:i2, j])
        spec_bgd.EuEu[i, j] = np.median(spec_cln.EuEu[i1:i2, j])
        spec_bgd.EvEv[i, j] = np.median(spec_cln.EvEv[i1:i2, j])
        spec_bgd.EwEw[i, j] = np.median(spec_cln.EwEw[i1:i2, j])
EE_bgd_med = np.median(spec_bgd.EE, axis=0)
EuEu_bgd_med = np.median(spec_bgd.EuEu, axis=0)
EvEv_bgd_med = np.median(spec_bgd.EvEv, axis=0)
EwEw_bgd_med = np.median(spec_bgd.EwEw, axis=0)

# Diff
spec_dif.EE = spec_org.EE - spec_bgd.EE
spec_dif.EuEu = spec_org.EuEu - spec_bgd.EuEu
spec_dif.EvEv = spec_org.EvEv - spec_bgd.EvEv
spec_dif.EwEw = spec_org.EwEw - spec_bgd.EwEw
EE_dif_med = np.median(spec_dif.EE, axis=0)
EuEu_dif_med = np.median(spec_dif.EuEu, axis=0)
EvEv_dif_med = np.median(spec_dif.EvEv, axis=0)
EwEw_dif_med = np.median(spec_dif.EwEw, axis=0)

# 2D
EE_cln_2d   = spec_cln.EE.transpose() 
EuEu_cln_2d = spec_cln.EuEu.transpose()
EvEv_cln_2d = spec_cln.EvEv.transpose()
EwEw_cln_2d = spec_cln.EwEw.transpose()

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
# Y-range
p_min = p_min0-2;   p_max = p_max0
# X-range
f_min = f_min0;  f_max = f_max0
#if f_mode == 0:
#    f_min = f_min0;  f_max = 5000
#f_mode = 0;   f_min = 2000;  f_max = 5000;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

print(EE_med.shape, freq_1d_msk0.shape)
ax1.plot(freq_1d_msk0, EE_med, '-k', linewidth=.5, label='EE ave')
ax2.plot(freq_1d_msk0, EuEu_med, '-k', linewidth=.5, label='EuEu ave')
ax3.plot(freq_1d_msk0, EvEv_med, '-k', linewidth=.5, label='EvEv ave')
ax4.plot(freq_1d_msk0, EwEw_med, '-k', linewidth=.5, label='EwEw ave')
ax1.plot(freq_1d_msk0, EE_cln_med, '-r', linewidth=.5, label='EE f-clean')
ax2.plot(freq_1d_msk0, EuEu_cln_med, '-r', linewidth=.5, label='EuEu f-clean')
ax3.plot(freq_1d_msk0, EvEv_cln_med, '-r', linewidth=.5, label='EvEv f-clean')
ax4.plot(freq_1d_msk0, EwEw_cln_med, '-r', linewidth=.5, label='EwEw f-clean')
ax1.plot(freq_1d_msk0, EE_bgd_med, '-g', linewidth=.5, label='EE BG')
ax2.plot(freq_1d_msk0, EuEu_bgd_med, '-g', linewidth=.5, label='EuEu BG')
ax3.plot(freq_1d_msk0, EvEv_bgd_med, '-g', linewidth=.5, label='EvEv BG')
ax4.plot(freq_1d_msk0, EwEw_bgd_med, '-g', linewidth=.5, label='EwEw BG')
ax1.plot(freq_1d_msk0, EE_dif_med, '-b', linewidth=.5, label='EE Diff')
ax2.plot(freq_1d_msk0, EuEu_dif_med, '-b', linewidth=.5, label='EuEu Diff')
ax3.plot(freq_1d_msk0, EvEv_dif_med, '-b', linewidth=.5, label='EvEv Diff')
ax4.plot(freq_1d_msk0, EwEw_dif_med, '-b', linewidth=.5, label='EwEw Diff')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

# Peak
from scipy.signal import find_peaks
EE_dif_med_peaks,_ = find_peaks(EE_dif_med, height=3.0)
EuEu_dif_med_peaks,_ = find_peaks(EuEu_dif_med, height=3.0)
EvEv_dif_med_peaks,_ = find_peaks(EvEv_dif_med, height=3.0)
EwEw_dif_med_peaks,_ = find_peaks(EwEw_dif_med, height=3.0)
ax1.plot(freq_1d_msk0[EE_dif_med_peaks], EE_dif_med[EE_dif_med_peaks], '.r', label='EE Diff peak')
ax2.plot(freq_1d_msk0[EuEu_dif_med_peaks], EuEu_dif_med[EuEu_dif_med_peaks], '.r', label='EuEu Diff peak')
ax3.plot(freq_1d_msk0[EvEv_dif_med_peaks], EvEv_dif_med[EvEv_dif_med_peaks], '.r', label='EvEv Diff peak')
ax4.plot(freq_1d_msk0[EwEw_dif_med_peaks], EwEw_dif_med[EwEw_dif_med_peaks], '.r', label='EwEw Diff peak')

# Label
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax4.set_xlabel('Frequency [kHz]')
#
date1 = data.epoch[n_sweep1];  date1 = date1.strftime('First: %Y/%m/%d %R:%S   ')
date2 = data.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = data.epoch[n_sweep3];  date3 = date3.strftime('Last: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
# xlim=[8150, 8250]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)

# range: Y-axis
#ylim=[p_min*10, p_max*10]
ylim=[-10, 50]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec_med.png'
    fig.savefig(png_fname)

"""
# Peak
print("[First]")
peak_E = np.ravel(spec.EE[n_sweep1]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print("[Mid]")
peak_E = np.ravel(spec.EE[n_sweep2]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print("[Last]")
peak_E = np.ravel(spec.EE[n_sweep3]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
"""

# SID-3 emulations